# To Do:

- Clean data further.
- Think about breaking the data into two separate categories (longer versus shorter) and use LSTM cells for longer but recurrent for shorter. Skip-layers?
- Do we want to add wars before 1812?
- Bag of Words (word freq)

In [228]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import re

import os
os.chdir('/Users/wyattscott/Documents/DS6050/Project')

In [229]:
df = pd.read_csv("presidential_speeches.csv", usecols=['Date', 'President', 'Party', 'Transcript'])

In [230]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [231]:
df_wars = pd.DataFrame({
    'War_Name': ['First Barbary Wars',
                 'War of 1812',
                 'Indian Wars',
                 'Mexican-American War',
                 'Spanish-American War', 
                 'Mexican Border Wars', 
                 'World War 1', 
                 'World War 2', 
                 'Korean War', 
                 'Vietnam War', 
                 'Persian Gulf War', 
                 'OEF', 
                 'OFS', 
                 'OES', 
                 'OIF', 
                 'OND', 
                 'OIR'
                ],
    'Start_Date': ['1801-05-01',
                   '1812-06-18',
                   '1817-01-01',
                   '1846-04-25',
                   '1898-04-21',
                   '1916-05-09',
                   '1917-04-06',
                   '1941-12-07',
                   '1950-06-25',
                   '1964-08-05',
                   '1990-08-02',
                   '2001-10-07',
                   '2015-01-01',
                   '2021-10-01',
                   '2003-03-17',
                   '2010-11-01',
                   '2014-10-15'
                  ],
    'End_Date': ['1805-06-10',
                 '1815-02-18',
                 '1898-12-31',
                 '1848-02-02',
                 '1903-07-15',
                 '1917-04-05',
                 '1918-11-11',
                 '1946-12-31',
                 '1955-01-31',
                 '1975-05-07',
                 '1991-04-06',
                 '2014-12-28',
                 '2021-08-31',
                 '2024-02-20',
                 '2010-08-31',
                 '2011-12-15',
                 '2024-02-20'
                ],
})

df_wars['Start_Date'] = pd.to_datetime(df_wars['Start_Date'], format='%Y-%m-%d')
df_wars['End_Date'] = pd.to_datetime(df_wars['End_Date'], format='%Y-%m-%d')

In [232]:
# Filter speeches df for Date > 1814
df = df[df['Date'] >= ('1798-01-01')].copy()

In [233]:
# Create a new column called "War" in df and initialize it with zeros
df['War'] = 0

# Iterate over each row in df
for index, row in df.iterrows():
    # Get the current date from the row
    current_date = row['Date']
    
    # Check if the current date falls within 6 months before the start date of any war in df_wars
    for _, war_row in df_wars.iterrows():
        start_date = war_row['Start_Date']
        
        # Calculate 1yr before the start date of the war
        six_months_before = start_date - pd.DateOffset(years=1)
        
        # If the current date falls 6 months before the start date of the war, set "War" column to 1
        if current_date >= six_months_before and current_date <= start_date:
            df.at[index, 'War'] = 1
            break

In [234]:
df.head()

,Date,President,Party,Transcript,War
24,1798-03-23,John Adams,Federalist,As the safety and prosperity of nations ultima...,0
25,1798-12-08,John Adams,Federalist,Gentlemen of the Senate and Gentlemen of the H...,0
26,1799-12-03,John Adams,Federalist,It is with peculiar satisfaction that I meet t...,0
27,1799-12-19,John Adams,Federalist,Gentlemen of the Senate and Gentlemen of the H...,0
28,1800-05-21,John Adams,Federalist,Whereas the late wicked and treasonable insurr...,1


In [235]:
df.isnull().sum()

Date          0
President     0
Party         0
Transcript    1
War           0
dtype: int64

In [236]:
df = df.dropna(axis=0).reset_index(drop=True)

In [237]:
df.isnull().sum()

Date          0
President     0
Party         0
Transcript    0
War           0
dtype: int64

In [238]:
# Additional words to remove
additional_stopwords = ['united states', 'states', 'government', 'president', 'americans', 'america']

# Combine NLTK stopwords with additional stopwords
stop_words = set(stopwords.words('english') + additional_stopwords)

In [239]:
# Clean transcript for President's name (when POTUS is author)
def clean_transcript(row):
    '''
    Clean up the transcript by removing instances of the president's name, stop words, and punctuation.

    Parameters
    ----------
    row : pandas.Series
        A row from the DataFrame containing 'President' and 'Transcript' columns.

    Returns
    -------
    str
        The cleaned transcript with instances of the president's name, stop words, and punctuation removed.

    Notes
    -----
    This function iterates over each row in the DataFrame and removes the president's
    name from the transcript text. It converts both the president's name and the transcript
    text to lowercase to ensure case-insensitive matching. Additionally, it removes common
    English stop words, any additional stop words specified, and punctuation.
    '''
    president = row['President'].lower()
    transcript = str(row['Transcript'])

    # Remove floating-point numbers and integers
    transcript = re.sub(r'\b\d+(?:\.\d+)?\s+', '', transcript)
    
    # Convert transcript to lowercase
    transcript = transcript.lower()

    # Remove president's name
    transcript = transcript.replace(president, '').strip()

    # Remove punctuation using regular expressions
    transcript = re.sub(r'[^\w\s]', '', transcript)

    # Tokenize transcript
    tokens = word_tokenize(transcript)

    # Remove stop words
    cleaned_tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into a string
    cleaned_transcript = ' '.join(cleaned_tokens)

    return cleaned_transcript
# Apply the cleaning function to each row
df['Transcript'] = df.apply(clean_transcript, axis=1)

In [240]:
df.head()

,Date,President,Party,Transcript,War
0,1798-03-23,John Adams,Federalist,safety prosperity nations ultimately essential...,0
1,1798-12-08,John Adams,Federalist,gentlemen senate gentlemen house representativ...,0
2,1799-12-03,John Adams,Federalist,peculiar satisfaction meet 6th congress united...,0
3,1799-12-19,John Adams,Federalist,gentlemen senate gentlemen house representativ...,0
4,1800-05-21,John Adams,Federalist,whereas late wicked treasonable insurrection a...,1


In [241]:
df.isnull().sum()

Date          0
President     0
Party         0
Transcript    0
War           0
dtype: int64

In [242]:
df = df.dropna(axis=0).reset_index(drop=True)

In [243]:
df.isnull().sum()

Date          0
President     0
Party         0
Transcript    0
War           0
dtype: int64

In [247]:
# Save to new csv for later use
df.to_csv('Speeches_War_Clean.csv', index=False)